In [3]:
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Load data and preprocess abit

In [4]:
train_data = pd.read_csv("train AG_news.csv")
test_data = pd.read_csv("test AG_news.csv")

In [5]:
train_data.head()

,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [6]:
print(f"{train_data.shape=}\n{test_data.shape=}") 

train_data.shape=(120000, 3)
test_data.shape=(7600, 3)


In [7]:
train_data["full_text"] = train_data.Title + "<SEP>" + train_data.Description
train_data.drop(columns= ["Title", "Description"], inplace= True)
test_data["full_text"] = test_data.Title + "<SEP>" + test_data.Description
test_data.drop(columns= ["Title", "Description"], inplace= True)
train_data.head()

,Class Index,full_text
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."


In [ ]:
all_text = "".join(train_data.full_text.astype(str).tolist())
chars = sorted(list(set(all_text)))
special_tokens = ["<PAD>", "<SEP>", "<UNK>"]
chars = chars + special_tokens
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
def encode(text):
    output = []
    i = 0
    while i < len(text):
        if text[i:i+5] == "<SEP>":
            output.append(stoi["<SEP>"])
            i += 5
        else:
            output.append(stoi.get(text[i], stoi["<UNK>"]))
            i += 1
    return output
decode = lambda x: [itos[i] for i in x]


In [26]:
train_data

,Class Index,full_text
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."
...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...
119996,2,Renteria signing a top-shelf deal<SEP>Red Sox ...
119997,2,Saban not going to Dolphins yet<SEP>The Miami ...
119998,2,Today's NFL games<SEP>PITTSBURGH at NY GIANTS ...


In [25]:
train_data.reset_index(drop = True)

,Class Index,full_text
0,3,Wall St. Bears Claw Back Into the Black (Reute...
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...
4,3,"Oil prices soar to all-time record, posing new..."
...,...,...
119995,1,Pakistan's Musharraf Says Won't Quit as Army C...
119996,2,Renteria signing a top-shelf deal<SEP>Red Sox ...
119997,2,Saban not going to Dolphins yet<SEP>The Miami ...
119998,2,Today's NFL games<SEP>PITTSBURGH at NY GIANTS ...


# Build model